In [1]:
import numpy as np
import tensorflow as tf
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
import os
from tensorflow import keras 
import glob as gb
import pandas as pd
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D ,LeakyReLU
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
#Configuramos los datos de entrenamiento y validación
IMG_SIZE = (274, 274) # resolution
directory = 'C:/Users/jofra/Desktop/Leaves'
BATCH_SIZE = 128
train_ds = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             labels='inferred',
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='training',
                                             color_mode='rgb',
                                             seed=42)
val_ds = image_dataset_from_directory(directory,
                                             shuffle=True,
                                             labels='inferred',
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE,
                                             validation_split=0.2,
                                             subset='validation',
                                             color_mode='rgb',
                                             seed=42)

Found 3300 files belonging to 3 classes.
Using 2640 files for training.
Found 3300 files belonging to 3 classes.
Using 660 files for validation.


In [4]:
#definimos el valor del numero pi
pi=3.14159265358979323846

In [5]:
#Podemos ver que los datos estan divididos en tres conjuntos, uno para cada clasificacion
class_names = train_ds.class_names
class_names

['Healthy', 'aculus_olearius', 'train4']

In [6]:
#Definimos como vamos a hacer nuestro aumento de datos
data_augmentation = tf.keras.Sequential([
  layers.RandomRotation(2*pi,fill_mode='constant',seed=68),
  layers.RandomFlip(mode='horizontal_and_vertical', seed=89),
  layers.RandomBrightness(factor=[-0.2,0.2],value_range=[0.0,1.0],seed=47)
  ])

In [7]:
#Vemos la forma de nuestros datos
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(128, 274, 274, 3)
(128,)


In [8]:
#Este bloque sirve para optimizar el almacenamiento en CPU
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:

#Definimos la arquitectura de la red neuronal
modeloCNN_augmentation = tf.keras.models.Sequential([
  layers.Rescaling(1./255, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
  data_augmentation,
  tf.keras.layers.Conv2D(16, (5,5), activation='relu'),
  tf.keras.layers.Conv2D(16, (5,5), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
  tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (5,5), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(128, (5,5), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(256, (5,5), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(2000, activation='relu'),
  tf.keras.layers.Dense(750, activation='relu'),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(3, activation='sigmoid')
])


In [10]:

#Compilamos el modelo
modeloCNN_augmentation.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])


In [11]:
#Cargamos tensorboard
from tensorflow.keras.callbacks import TensorBoard

In [12]:
#Escogemos la métrica que vamos a monitorear
metric_to_monitor = 'val_accuracy'

# Crear el objeto ModelCheckpoint para guardar el modelo
checkpoint = ModelCheckpoint('best_augmentation.h5', monitor=metric_to_monitor, save_best_only=True, mode='max')



In [13]:
#Entrenamos el modelo
tensorboardCNN_augmentation = TensorBoard(log_dir='logs/cnn_augmentation')
modeloCNN_augmentation.fit(train_ds,
                validation_data=val_ds,
                epochs=200,
                callbacks=[tensorboardCNN_augmentation,checkpoint])

Epoch 1/200


C:\Users\jofra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


21/21 [==============================] - 200s 9s/step - loss: 1.1042 - accuracy: 0.4330 - val_loss: 0.9594 - val_accuracy: 0.4379
Epoch 2/200
21/21 [==============================] - 186s 9s/step - loss: 0.9616 - accuracy: 0.4523 - val_loss: 0.8389 - val_accuracy: 0.5545
Epoch 3/200
21/21 [==============================] - 180s 9s/step - loss: 0.8458 - accuracy: 0.5670 - val_loss: 0.7369 - val_accuracy: 0.6091
Epoch 4/200
21/21 [==============================] - 187s 9s/step - loss: 0.7735 - accuracy: 0.6045 - val_loss: 0.6933 - val_accuracy: 0.6273
Epoch 5/200
21/21 [==============================] - 193s 9s/step - loss: 0.7465 - accuracy: 0.6170 - val_loss: 0.6917 - val_accuracy: 0.6348
Epoch 6/200
21/21 [==============================] - 194s 9s/step - loss: 0.7627 - accuracy: 0.6163 - val_loss: 0.6784 - val_accuracy: 0.6485
Epoch 7/200
 9/21 [===========>..................] - ETA: 1:47 - loss: 0.7347 - accuracy: 0.6319

KeyboardInterrupt: 

In [14]:
#Hacemos predicciones sobre el conjunto de validación
predCNN=modeloCNN_augmentation.predict(val_ds)

6/6 [==============================] - 7s 1s/step


In [15]:
#Nos quedamos con la clase que tenga mayor probabilidad
CNNpred = tf.argmax(predCNN, axis=1)

In [16]:
true_categories = tf.concat([y for x, y in val_ds], axis=0)

In [17]:
#Matriz de confusión
tf.math.confusion_matrix(
    true_categories,
    CNNpred,
    num_classes=None,
    weights=None,
    dtype=tf.dtypes.int32,
    name=None
)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[155,  12,   8],
       [148,  17,  31],
       [ 49,   3, 237]])>

### Cargamos el modelo ya entrenado

In [18]:
#Cargamos el modelo ya entrenado
model_loaded = keras.models.load_model('CNN_augmented.h5')

In [19]:
#Evaluamos la precision del modelo
model_loaded.evaluate(val_ds)

6/6 [==============================] - 8s 1s/step - loss: 0.1079 - accuracy: 0.9727


[0.10793013125658035, 0.9727272987365723]

In [20]:
#Hacemos las predicciones
prednueva=model_loaded.predict(val_ds)

6/6 [==============================] - 8s 1s/step


In [21]:
#Nos quedamos con la clase que tenga mayor probabilidad
npred = tf.argmax(prednueva, axis=1)

In [22]:
true_categories = tf.concat([y for x, y in val_ds], axis=0)

In [23]:
#Creamos la matriz de confusión
tf.math.confusion_matrix(
    true_categories,
    npred,
    num_classes=None,
    weights=None,
    dtype=tf.dtypes.int32,
    name=None
)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[167,   4,   4],
       [  0, 195,   1],
       [  3,   7, 279]])>

In [24]:
#Vemos la arquitecura del modelo
model_loaded.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_8 (Rescaling)     (None, 274, 274, 3)       0         
                                                                 
 sequential (Sequential)     (None, 274, 274, 3)       0         
                                                                 
 conv2d_27 (Conv2D)          (None, 270, 270, 16)      1216      
                                                                 
 conv2d_28 (Conv2D)          (None, 266, 266, 16)      6416      
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 133, 133, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_29 (Conv2D)          (None, 129, 129, 32)      12832     
                                                     

In [25]:
#Ejecutar tensorboard e indicarle que lea la carpeta "logs" para ver las graficas
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.
